In [2]:
from youtube_api import YouTubeDataAPI
import pandas as pd
from apikey import api_key
from apikey import username
from apikey import password
import requests as rq
import re
import isodate
from sqlalchemy import create_engine

In [2]:
# creating the connection to upload the table to sql database

driver = 'mysql+pymysql'
ip = 'barcelona-db.cyxhqbnhiohl.eu-west-3.rds.amazonaws.com'
db = 'project3'
connection_string  = f'{driver}://{username}:{password}@{ip}/{db}'

In [3]:
engine = create_engine(connection_string)

In [25]:
# We picked 3 independent creators who talked about how the new algorithm affected them:
# Jaime Altozano = UCa3DVlGH2_QhvwuWlPa6MDQ
# Ter = UCCNgRIfWQKZyPkNvHEzPh7Q
# Cocina Veggikinga
# We want to check their data and compare it to the Top3
medium3 = ['UCa3DVlGH2_QhvwuWlPa6MDQ', 'UCCNgRIfWQKZyPkNvHEzPh7Q', 'UCLj5tilgmgH862bkmJnOo4g']

In [27]:
# Creating the access to the Youtube API
yt = YouTubeDataAPI(api_key)

In [28]:
# Defining a function to get videos given the name of a channel, limiting at 50.

def getVideos(channel):
    return yt.search(channel_id=channel, max_results=50)

In [29]:
channelVideos = [] # this will be the list where we will append all the data got from the API

In [30]:
# we loop over each channel in 'channels' list to get the videos using the function.
for channel in medium3:
    try:
        channelVideos.append(getVideos(channel))
    except:
        pass

In [31]:
len(channelVideos)

3

In [32]:
channelVideos

[[{'video_id': 'faBivRMi2LY',
   'channel_title': 'Jaime Altozano',
   'channel_id': 'UCa3DVlGH2_QhvwuWlPa6MDQ',
   'video_publish_date': datetime.datetime(2018, 3, 16, 11, 38, 41),
   'video_title': '¿Por qué medimos el RITMO así? Compás de 4/4, 2/4, 6/8, 3/4... | Jaime Altozano',
   'video_description': 'Por qué se dicen cosas como "en este compás cambia el compás"? Cómo se mide el ritmo en música? El ritmo es una manera de crear patrones en el tiempo.',
   'video_category': None,
   'video_thumbnail': 'https://i.ytimg.com/vi/faBivRMi2LY/hqdefault.jpg',
   'collection_date': datetime.datetime(2019, 11, 14, 14, 34, 4, 835679)},
  {'video_id': 'e9ay6zMICw8',
   'channel_title': 'Jaime Altozano',
   'channel_id': 'UCa3DVlGH2_QhvwuWlPa6MDQ',
   'video_publish_date': datetime.datetime(2019, 4, 25, 18, 57, 47),
   'video_title': 'Explicación Definitiva de los MODOS Jónico, Dórico, Frigio, Lidio, Mixolidio, Eólico y Locrio',
   'video_description': 'Qué son los modos Jónico, Dórico, Frigio,

In [33]:
# Retrieving a list of video_ids
channelName = []
videoIds = []
videoPublishDate = []
for i in range(len(channelVideos)):
    for video in channelVideos[i]:
        channelName.append(video.get('channel_title'))
        videoIds.append(video.get('video_id'))
        videoPublishDate.append(video.get('video_publish_date').strftime("%m/%d/%Y"))

In [34]:
Medium3_Id_df = pd.DataFrame({'Channel': channelName, 'Video_Id': videoIds, 'Published':videoPublishDate})

In [35]:
# Creating a csv file to don't need to query again to the API
Medium3_Id_df.to_csv('./data/Medium3_Id_dirty.csv', sep='\t', index = False)

In [37]:
Medium3_Id_df['Channel'].unique()

array(['Jaime Altozano', 'Ter', 'Cocina Veggikinga'], dtype=object)

In [38]:
cleaned_video_id = list(Medium3_Id_df['Video_Id'])

In [39]:
# preparing the url to get the video details looping through the list of video ids.
url_a = 'https://www.googleapis.com/youtube/v3/videos?part=contentDetails&id='
url_b = f'&key={api_key}'

In [40]:
# looping to get the video details from each video Id.
videoDetails = []

for elem in cleaned_video_id:
    videoDetails.append(rq.get(url_a + elem + url_b).json())

In [41]:
# looping over videoDetails to get the duration of each video
durations = []

for i in range(len(videoDetails)):
    durations.append(videoDetails[i].get('items')[0].get('contentDetails').get('duration'))

In [42]:
Medium3_Id_df['Duration']=durations

In [43]:
Medium3_Id_df['Duration'] = Medium3_Id_df['Duration'].apply(lambda x: isodate.parse_duration(x).seconds)

In [44]:
Medium3_Id_df.reset_index(drop = True, inplace = True)

In [45]:
Medium3_Id_df

,Channel,Video_Id,Published,Duration
0,Jaime Altozano,faBivRMi2LY,03/16/2018,371
1,Jaime Altozano,e9ay6zMICw8,04/25/2019,576
2,Jaime Altozano,Mb2gw7O43ww,10/13/2018,212
3,Jaime Altozano,ziJuNQcJn24,06/30/2018,213
4,Jaime Altozano,BRBTCIK_9_g,02/01/2019,763
...,...,...,...,...
145,Cocina Veggikinga,rmW626L1buU,09/20/2019,613
146,Cocina Veggikinga,Xbk4BVInzEg,06/14/2019,2203
147,Cocina Veggikinga,523nHbXi1fs,11/01/2019,422
148,Cocina Veggikinga,g_WYCyxYjVQ,08/20/2019,920


In [46]:
Medium3_Id_df.to_csv('./data/Medium3_durations.csv', sep='\t', index = False)

In [85]:
# Importing rows to database table
from sqlalchemy import exc
num_rows = len(Top3_df)
#Iterate one row at a time
for i in range(num_rows):
    try:
       #Try inserting the row
       Top3_df.iloc[i:i+1].to_sql(name="video_table",con = engine,if_exists = 'append',index=False)
    except exc.IntegrityError:
       #Ignore duplicates
       pass